# Operationalizing training with KFP

In [1]:
import kfp
from kfp import gcp
import json

In [11]:
PROJECT_ID = !gcloud config list project --format "value(core.project)"
PROJECT_ID = PROJECT_ID[0]
REGION = 'us-central1'
MASTER_IMAGE_URI = 'gcr.io/jk-demo1/octane-regression-training:latest'
JOB_DIR = "gs://jk-demo-jobdir/"
ARGS = json.dumps([
    '--data-path', 'gs://jk-demo-datasets/gasdata/training.csv',
    '--n-features-options', '[2,4,6]', 
    '--l2-reg-options', '[0.1,0.2,0.3,0.5]'
])
    
TRAIN_COMPONENT_SPEC="https://raw.githubusercontent.com/kubeflow/pipelines/master/components/gcp/ml_engine/train/component.yaml"
DEPLOY_COMPONENT_SPEC=""
JOB_ID_PREFIX="JOB"

## Define the pipeline

In [12]:
mlengine_train_op = kfp.components.load_component_from_url(TRAIN_COMPONENT_SPEC)

In [13]:
@kfp.dsl.pipeline(
    name="CAIP_training",
    description="Cloud AI Platform Training and Deployment")
def pipeline(
    project_id=PROJECT_ID,
    region=REGION,
    master_image_uri=MASTER_IMAGE_URI,
    job_dir=JOB_DIR,
    job_id_prefix=JOB_ID_PREFIX,
    args=ARGS):
    
  train_task = mlengine_train_op(
      project_id=project_id,
      region=region,
      args=args,
      job_dir=job_dir,
      master_image_uri=master_image_uri,
   ).apply(gcp.use_gcp_secret('user-gcp-sa'))
  

## Compile the pipeline

In [14]:
pipeline_filename = pipeline._pipeline_name + '.zip'
kfp.compiler.Compiler().compile(pipeline, pipeline_filename)


## Submit the run
### Fetch GKE credentials

In [15]:
CLUSTER_NAME = "kf-gke-cluster"
ZONE = "us-west1-a"

!gcloud container clusters get-credentials $CLUSTER_NAME --zone $ZONE

Fetching cluster endpoint and auth data.
kubeconfig entry generated for kf-gke-cluster.


### Create a KFP experiment

In [16]:
EXPERIMENT_NAME = "CAIP_Training_Deployment"

client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

### Submit a run

In [17]:
run_name =  'run1'
arguments = {}
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)